# Code For simulation LFPy

In [ ]:
import LFPy
import numpy as np
import os
import urllib2
import zipfile
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.collections import PolyCollection
%matplotlib inline

#morphname = 'patdemo/cells/j4a'
morphname = 'patdemo/cells/03b_pyramidal2aACC'

def setcellparams(morphname,Par):
    # Define cell parameters
    cell_parameters = {          # various cell parameters,
        'morphology' : morphname + '.hoc', # Mainen&Sejnowski, 1996
        'rm' : 30000.,      # membrane resistance
        'cm' : 1.0,         # membrane capacitance
        'Ra' : 150,        # axial resistance
        'v_init' : -65.,    # initial crossmembrane potential
        'e_pas' : -65.,     # reversal potential passive mechs
        'passive' : True,   # switch on passive mechs
        'nsegs_method' : 'lambda_f',
        'lambda_f' : 100.,
        'timeres_NEURON' : 2.**-3,   # [ms] dt's should be in powers of 2 for both,
        'timeres_python' : 2.**-3,   # need binary representation
        'tstartms' : 0.,    # start time of simulation, recorders start at t=0
        'tstopms' : 50.,   # stop simulation at 200 ms. These can be overridden
                        # by setting these arguments i cell.simulation()
    }

    # Create cell
    cell = LFPy.Cell(**cell_parameters)

    # Align cell
    cell.set_rotation(x=4.99, y=-4.33, z=3.14)
    # cell.set_rotation(x=cellrotation[0], y=cellrotation[1], z=cellrotation[2])

    # Define synapse parameters
    synapse_parameters = {
        'idx' : cell.get_closest_idx(x=0., y=0., z=0.),
        'e' : 0.,                   # reversal potential
        'syntype' : 'ExpSyn',       # synapse type
        'tau' : Par[0],                 # synaptic time constant
        # tau ~ 5
        'weight' : Par[1],            # synaptic weight
        # weight ~ .001
        'record_current' : True,    # record synapse current
    }

    # Create synapse and set time of synaptic input (input at 20ms)
    synapse = LFPy.Synapse(cell, **synapse_parameters)
    synapse.set_spike_times(np.array([0]))

    return cell


def simulateLFP(cell):
    
    # Create a grid of measurement locations, in (mum)
    X, Y, Z = np.mgrid[-200:201:10,-200:200:10, -300:401:10]
    #Y = np.zeros(X.shape)
    # Y = np.ones(X.shape)*yval
    
    # Define electrode parameters
    grid_electrode_parameters = {
        'sigma' : 0.3,      # extracellular conductivity
        'x' : X.flatten(),  # electrode requires 1d vector of positions
        'y' : Y.flatten(),
        'z' : Z.flatten(),
        'method' : 'pointsource'
    }
    

    # Define electrode parameters
    #point_electrode_parameters = {
    #    'sigma' : 0.3,  # extracellular conductivity
    #    'x' : np.array([-130., -220.]),
    #    'y' : np.array([   0.,    0.]),
    #    'z' : np.array([   0.,  700.]),
    #}
    
    
    cell.simulate(rec_imem=True,rec_isyn=True)

    # Create electrode objects
    grid_electrode = LFPy.RecExtElectrode(cell,**grid_electrode_parameters)
    #point_electrode = LFPy.RecExtElectrode(cell,**point_electrode_parameters)

    # Calculate LFPs
    grid_electrode.calc_lfp()
    #point_electrode.calc_lfp()
    
    fullLFP = grid_electrode.LFP
    
    return fullLFP
    
#write data to csv file


T = np.mgrid[5:20:20]
W = np.mgrid[.001:.005:.008]


idx = np.arange(0,T.size)
idy = np.arange(0,W.size)


for i in idx:
    for j in idy:
                 Par = np.array([T[i],W[j]])
                 cell = setcellparams(morphname,Par)
                 fullLFP = simulateLFP(cell)
            
                 # save results of simulation
                 sz = fullLFP.shape
                 nelec = sz[0]
                 listelec = np.arange(1,nelec)
            
                 fullLFP_pd = pd.DataFrame(fullLFP)
                 strname = 'LFData_T_' + str(T[i]) + '_W_' + str(W[j]) + '.csv'
                 fullLFP_pd.to_csv(strname)           
                 print str(i)+'_'+ str(j)